In [1]:
#Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from simpledbf import Dbf5

#Set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:.2f}'.format

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
df = Dbf5('nbac_1986_to_2022_20230630.dbf').to_dataframe()
df.to_csv('nbac_raw.csv', index=False)

In [3]:
df['AFSDATE'].isna().sum()

9202

In [4]:
df['AFSDATE'] = df['AFSDATE'].fillna(df['SDATE'])
df['AFSDATE'].isna().sum()

6695

In [5]:
df['AFSDATE'] = pd.to_datetime(df['AFSDATE'])

In [7]:
df_formatted = df[df['YEAR'] > 1997][['AGENCY','AFSDATE','POLY_HA']].rename(columns={'AGENCY':'province','AFSDATE':'date','POLY_HA':'size_ha'})

In [8]:
#Create Provincial data frames
df_bc = df_formatted[df_formatted['province'] == 'BC'].rename(columns={'size_ha':'bc_size_ha'})
df_ab = df_formatted[df_formatted['province'] == 'AB'].rename(columns={'size_ha':'ab_size_ha'})
df_on = df_formatted[df_formatted['province'] == 'ON'].rename(columns={'size_ha':'on_size_ha'})
df_mb = df_formatted[df_formatted['province'] == 'MB'].rename(columns={'size_ha':'mb_size_ha'})
df_qc = df_formatted[df_formatted['province'] == 'QC'].rename(columns={'size_ha':'qc_size_ha'})
df_sk = df_formatted[df_formatted['province'] == 'SK'].rename(columns={'size_ha':'sk_size_ha'})
df_nt = df_formatted[df_formatted['province'] == 'NT'].rename(columns={'size_ha':'nt_size_ha'})
df_nb = df_formatted[df_formatted['province'] == 'NB'].rename(columns={'size_ha':'nb_size_ha'})
df_ns = df_formatted[df_formatted['province'] == 'NS'].rename(columns={'size_ha':'ns_size_ha'})
df_yt = df_formatted[df_formatted['province'] == 'YT'].rename(columns={'size_ha':'yt_size_ha'})
df_nl = df_formatted[df_formatted['province'] == 'NL'].rename(columns={'size_ha':'nl_size_ha'})

In [9]:
df_ab.to_csv('ab_clean.csv', index=False)
df_bc.to_csv('bc_clean.csv', index=False)
df_on.to_csv('on_clean.csv', index=False)
df_mb.to_csv('mb_clean.csv', index=False)
df_qc.to_csv('qc_clean.csv', index=False)
df_sk.to_csv('sk_clean.csv', index=False)
df_nt.to_csv('nt_clean.csv', index=False)
df_nb.to_csv('nb_clean.csv', index=False)
df_ns.to_csv('ns_clean.csv', index=False)
df_yt.to_csv('yt_clean.csv', index=False)
df_nl.to_csv('nl_clean.csv', index=False)

In [28]:
df_on

,province,date,on_size_ha
7027,ON,1998-07-06,3746.10
7030,ON,1998-06-25,1808.37
7035,ON,1998-07-06,526.48
7038,ON,1998-06-24,22273.58
7044,ON,1998-06-30,828.38
...,...,...,...
37581,ON,NaT,58.83
37931,ON,NaT,94.09
38118,ON,NaT,72.42
38232,ON,NaT,57.51


In [10]:
national = df_formatted.join(pd.get_dummies(df_formatted['province'],dtype='float').mul(df_formatted.size_ha, axis=0))
national = national.drop(columns=national.loc[:, 'PC-BA':'PC-YO'])
national = national.drop(columns=['province','size_ha'])
national = national.rename(columns={'AB':'ab_size_ha','BC':'bc_size_ha','MB':'mb_size_ha','NB':'nb_size_ha','NL':'nl_size_ha','NS':'ns_size_ha','NT':'nt_size_ha','NU':'nu_size_ha','ON':'on_size_ha','QC':'qc_size_ha','SK':'sk_size_ha','YT':'yt_size_ha'})
national = national.groupby('date').sum().reset_index()

In [110]:
national.to_csv('national_clean.csv',index=False)

In [111]:
national['date'] = pd.to_datetime(national['date'])
national.groupby(national.date.dt.year).agg({'ab_size_ha':'sum','bc_size_ha':'sum','mb_size_ha':'sum','nb_size_ha':'sum','nl_size_ha':'sum','ns_size_ha':'sum','nt_size_ha':'sum','on_size_ha':'sum','qc_size_ha':'sum','sk_size_ha':'sum','yt_size_ha':'sum'})

,ab_size_ha,bc_size_ha,mb_size_ha,nb_size_ha,nl_size_ha,ns_size_ha,nt_size_ha,on_size_ha,qc_size_ha,sk_size_ha,yt_size_ha
date,,,,,,,,,,,
1998,617923.90,73553.15,440785.23,0.00,32538.35,0.00,1267758.56,212228.23,395352.42,815686.01,304406.36
1999,93430.94,13494.59,233112.55,0.00,38498.01,697.78,438949.34,296584.37,138643.93,155469.33,170552.72
2000,7525.51,5899.60,116057.55,0.00,98517.55,146.82,157398.57,15341.60,57994.13,119178.88,7871.03
2001,138062.49,6145.57,59399.33,0.00,0.00,0.00,84717.65,11935.10,31655.83,136097.81,18196.82
2002,499525.41,4734.18,68749.32,0.00,0.00,0.00,41122.00,196279.83,1186271.58,674445.79,34231.50
2003,63639.60,220618.33,723585.00,0.00,30104.52,494.43,104693.08,333841.39,92037.52,96468.61,49418.78
2004,230080.16,180999.33,21383.92,0.00,923.82,0.00,384539.68,2425.48,4548.32,270544.36,1623657.79
2005,56204.79,25955.38,64616.77,375.27,20350.96,0.00,177671.43,48099.26,846134.27,189304.29,167465.27
2006,155587.88,114470.87,136768.90,0.00,2376.78,0.00,42995.97,124482.64,140351.65,1051138.46,90175.66


In [112]:
national_panel = df[df['YEAR'] > 1997][['AGENCY','YEAR','POLY_HA']].rename(columns={'AGENCY':'province','YEAR':'year','POLY_HA':'size_ha'})
national_panel = national_panel.groupby(['province','year']).agg({'size_ha':'sum'}).reset_index()
national_panel.to_csv('national_panel_clean.csv',index=False)

In [113]:
national_panel

,province,year,size_ha
0,AB,1998.00,633821.18
1,AB,1999.00,105838.14
2,AB,2000.00,13994.38
3,AB,2001.00,154969.58
4,AB,2002.00,503130.42
...,...,...,...
526,YT,2018.00,80654.05
527,YT,2019.00,272838.67
528,YT,2020.00,15940.68
529,YT,2021.00,133944.94


In [114]:
# test error_perc to stated numbers
df_ab['date'] = pd.to_datetime(df_ab['date'])
df_ab['Year'] = df_ab['date'].dt.year
validate = df_ab.groupby(df_ab['Year']).agg({'ab_size_ha':'sum'}).reset_index()
ab_reported = pd.read_csv('goahectaresburnedfromwildfirecsvv2.02015-07-30.csv').rename(columns={'Value':'reported'})
ab_reported = ab_reported[ab_reported['Variable'] != 'Number of Wildfires'][['Year','reported']]
validate = validate.merge(ab_reported, on='Year')
validate = validate.replace(',','', regex=True)
validate['reported'] = validate['reported'].astype('float')
validate['error_perc'] = abs(((validate['reported'] - validate['ab_size_ha']) / validate['reported'])*100)
validate['error_perc'].mean()

12.157457629941394

In [115]:
ab_reported

,Year,reported
0,2005,"60,657"
2,2006,"118,762"
4,2007,"103,649"
6,2008,"20,747"
8,2009,"66,945"
10,2010,"81,105"
12,2011,"805,900"
14,2012,"385,655"
16,2013,"19,572"
18,2014,"23,716"
